# Characterizing perturbation of cellular transcriptomes due to RNA exporter expression

Helper notebook for `transcriptome_stress.ipynb`. Here we run differential expression analysis using DESeq2 in R.

Note that running kernel must be an R kernel with DESeq2 installed.

# Imports

In [1]:
library("DESeq2")

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The follow

# Load data

In [2]:
# Load counts

infile <- "outs/transcriptome_stress/counts_clean_for_differential_expression.tsv"
counts <- read.csv(infile, sep="\t")

# Set row names
rownames(counts) <- counts$symbol_ensembl
counts$symbol_ensembl <- NULL

dim(counts)
head(counts)

[1] 30877     8

,Gag,GagMCP,GagZipMCP,EPN24MCP,No_Exporter_Psi,No_Exporter_MS2x8,NoDNA,Wildtype
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
EGFP,198592,98788,93438,62364,0,0,0,0
TSPAN6_ENSG00000000003,2691,2615,2802,2800,2464,3358,3184,2991
TNMD_ENSG00000000005,7,2,8,7,5,8,10,11
DPM1_ENSG00000000419,1514,1543,1533,1536,1263,1936,1291,1139
SCYL3_ENSG00000000457,535,614,653,625,522,686,614,557
C1orf112_ENSG00000000460,1251,1429,1486,1350,1005,1786,1077,1056


In [3]:
# Load metadata

infile <- "outs/transcriptome_stress/metadata_clean_for_differential_expression.tsv"
metadata <- read.csv(infile, sep="\t")

# Set row names
rownames(metadata) <- metadata$X
metadata$X <- NULL

# Convert columns to factors
metadata$label <- as.factor(metadata$label)
metadata$label_clean <- as.factor(metadata$label_clean)
metadata$condition_transfected <- as.factor(metadata$condition_transfected)
metadata$condition_exporter_indiv_no_exporter_pool <- as.factor(metadata$condition_exporter_indiv_no_exporter_pool)
metadata$condition_exporter_indiv_no_exporter_indiv <- as.factor(metadata$condition_exporter_indiv_no_exporter_indiv)
metadata$condition_exporter <- as.factor(metadata$condition_exporter)
metadata$condition_null <- as.factor(metadata$condition_null)

dim(metadata)
metadata

[1] 8 7

,label,condition_transfected,condition_exporter_indiv_no_exporter_pool,condition_exporter_indiv_no_exporter_indiv,condition_null,condition_exporter,label_clean
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
Gag,MMLV_Gag,transfected,exporter_Gag,exporter_Gag,exporter_Gag,exporter,Gag
GagMCP,Gag-MCP,transfected,exporter_GagMCP,exporter_GagMCP,exporter_GagMCP,exporter,GagMCP
GagZipMCP,GagZip-MCP,transfected,exporter_GagZipMCP,exporter_GagZipMCP,exporter_GagZipMCP,exporter,GagZipMCP
EPN24MCP,EPN24-MCP,transfected,exporter_EPN24MCP,exporter_EPN24MCP,exporter_EPN24MCP,exporter,EPN24MCP
No_Exporter_Psi,No_Exporter_Psi,transfected,no_exporter,no_exporter_Psi,no_exporter_Psi,no_exporter,No_Exporter_Psi
No_Exporter_MS2x8,No_Exporter_MS2x8,transfected,no_exporter,no_exporter_MS2,no_exporter_MS2,no_exporter,No_Exporter_MS2x8
NoDNA,No_DNA,untransfected,untransfected,untransfected,no_DNA,no_exporter,NoDNA
Wildtype,No_Transfection,untransfected,untransfected,untransfected,no_transfection,no_exporter,Wildtype


In [4]:
# It is absolutely critical that columns of count matrix and rows of metadata are in the same order.
# Confirm this is true.
all(rownames(metadata) == colnames(counts))

[1] TRUE

# Perform differential expression analysis

## Analysis for MMLV Gag (Gag + mCh-Psi versus mCh-Psi)

In [5]:
dds_no_exporter_indiv_Psi <- DESeqDataSetFromMatrix(countData = counts,
                                           colData = metadata,
                                           design = ~ condition_exporter_indiv_no_exporter_indiv)

dds_no_exporter_indiv_Psi$condition_exporter_indiv_no_exporter_indiv <- relevel(dds_no_exporter_indiv_Psi$condition_exporter_indiv_no_exporter_indiv, ref = "no_exporter_Psi")

In [6]:
dds_no_exporter_indiv_Psi <- DESeq(dds_no_exporter_indiv_Psi)  # perform analysis

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [7]:
resultsNames(dds_no_exporter_indiv_Psi)

[1] "Intercept"                                                                       
[2] "condition_exporter_indiv_no_exporter_indiv_exporter_EPN24MCP_vs_no_exporter_Psi" 
[3] "condition_exporter_indiv_no_exporter_indiv_exporter_Gag_vs_no_exporter_Psi"      
[4] "condition_exporter_indiv_no_exporter_indiv_exporter_GagMCP_vs_no_exporter_Psi"   
[5] "condition_exporter_indiv_no_exporter_indiv_exporter_GagZipMCP_vs_no_exporter_Psi"
[6] "condition_exporter_indiv_no_exporter_indiv_no_exporter_MS2_vs_no_exporter_Psi"   
[7] "condition_exporter_indiv_no_exporter_indiv_untransfected_vs_no_exporter_Psi"

In [8]:
# Gag vs no exporter (mCh-Psi)
res_no_exporter_indiv_Gag_vs_no_exporter <- results(dds_no_exporter_indiv_Psi, alpha=0.05, contrast=c("condition_exporter_indiv_no_exporter_indiv", "exporter_Gag", "no_exporter_Psi"))  # get results
res_no_exporter_indiv_Gag_vs_no_exporter <- res_no_exporter_indiv_Gag_vs_no_exporter[order(res_no_exporter_indiv_Gag_vs_no_exporter$padj),]  # sort by p value
summary(res_no_exporter_indiv_Gag_vs_no_exporter)
res_no_exporter_indiv_Gag_vs_no_exporter


out of 30785 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)       : 1, 0.0032%
LFC < 0 (down)     : 1, 0.0032%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



log2 fold change (MLE): condition_exporter_indiv_no_exporter_indiv exporter_Gag vs no_exporter_Psi 
Wald test p-value: condition exporter indiv no exporter indiv exporter Gag vs no exporter Psi 
DataFrame with 30877 rows and 6 columns
                         baseMean log2FoldChange     lfcSE      stat
                        <numeric>      <numeric> <numeric> <numeric>
EGFP                    56992.422      19.788085  2.048624  9.659206
HSPA6_ENSG00000173110     158.544      -3.261385  0.673296 -4.843909
mTagBFP2               334092.593      -0.767034  0.182738 -4.197443
MT-ND2_ENSG00000198763  23767.429       0.855777  0.212239  4.032138
TSPAN6_ENSG00000000003   2843.732      -0.126597  0.222562 -0.568818
...                           ...            ...       ...       ...
ERCC-00164                      0             NA        NA        NA
ERCC-00165                      0             NA        NA        NA
ERCC-00168                      0             NA        NA        NA
ERCC-0

## Analysis for Gag-MCP, GagZip-MCP, and EPN24-MCP (exporter + mCh-MS2x8 versus mCh-MS2x8)

In [9]:
dds_no_exporter_indiv_MS2 <- DESeqDataSetFromMatrix(countData = counts,
                                           colData = metadata,
                                           design = ~ condition_exporter_indiv_no_exporter_indiv)

dds_no_exporter_indiv_MS2$condition_exporter_indiv_no_exporter_indiv <- relevel(dds_no_exporter_indiv_MS2$condition_exporter_indiv_no_exporter_indiv, ref = "no_exporter_MS2")

In [10]:
dds_no_exporter_indiv_MS2 <- DESeq(dds_no_exporter_indiv_MS2)  # perform analysis

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [11]:
resultsNames(dds_no_exporter_indiv_MS2)

[1] "Intercept"                                                                       
[2] "condition_exporter_indiv_no_exporter_indiv_exporter_EPN24MCP_vs_no_exporter_MS2" 
[3] "condition_exporter_indiv_no_exporter_indiv_exporter_Gag_vs_no_exporter_MS2"      
[4] "condition_exporter_indiv_no_exporter_indiv_exporter_GagMCP_vs_no_exporter_MS2"   
[5] "condition_exporter_indiv_no_exporter_indiv_exporter_GagZipMCP_vs_no_exporter_MS2"
[6] "condition_exporter_indiv_no_exporter_indiv_no_exporter_Psi_vs_no_exporter_MS2"   
[7] "condition_exporter_indiv_no_exporter_indiv_untransfected_vs_no_exporter_MS2"

In [12]:
# GagMCP vs no exporter (mCh-MS2x8)
res_no_exporter_indiv_GagMCP_vs_no_exporter <- results(dds_no_exporter_indiv_MS2, alpha=0.05, contrast=c("condition_exporter_indiv_no_exporter_indiv", "exporter_GagMCP", "no_exporter_MS2"))  # get results
res_no_exporter_indiv_GagMCP_vs_no_exporter <- res_no_exporter_indiv_GagMCP_vs_no_exporter[order(res_no_exporter_indiv_GagMCP_vs_no_exporter$padj),]  # sort by p value
summary(res_no_exporter_indiv_GagMCP_vs_no_exporter)
res_no_exporter_indiv_GagMCP_vs_no_exporter


out of 30785 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)       : 1, 0.0032%
LFC < 0 (down)     : 0, 0%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



log2 fold change (MLE): condition_exporter_indiv_no_exporter_indiv exporter_GagMCP vs no_exporter_MS2 
Wald test p-value: condition exporter indiv no exporter indiv exporter GagMCP vs no exporter MS2 
DataFrame with 30877 rows and 6 columns
                          baseMean log2FoldChange     lfcSE      stat
                         <numeric>      <numeric> <numeric> <numeric>
EGFP                   56992.42223     19.3051201  2.048627  9.423443
TSPAN6_ENSG00000000003  2843.73150     -0.0900759  0.222107 -0.405552
TNMD_ENSG00000000005       7.14874     -1.7292661  2.689037 -0.643080
DPM1_ENSG00000000419    1457.65253     -0.0566254  0.264254 -0.214284
SCYL3_ENSG00000000457    597.22304      0.1107457  0.351383  0.315171
...                            ...            ...       ...       ...
ERCC-00164                       0             NA        NA        NA
ERCC-00165                       0             NA        NA        NA
ERCC-00168                       0             NA        NA

In [13]:
# GagZipMCP vs no exporter (mCh-MS2x8)
res_no_exporter_indiv_GagZipMCP_vs_no_exporter <- results(dds_no_exporter_indiv_MS2, alpha=0.05, contrast=c("condition_exporter_indiv_no_exporter_indiv", "exporter_GagZipMCP", "no_exporter_MS2"))  # get results
res_no_exporter_indiv_GagZipMCP_vs_no_exporter <- res_no_exporter_indiv_GagZipMCP_vs_no_exporter[order(res_no_exporter_indiv_GagZipMCP_vs_no_exporter$padj),]  # sort by p value
summary(res_no_exporter_indiv_GagZipMCP_vs_no_exporter)
res_no_exporter_indiv_GagZipMCP_vs_no_exporter


out of 30785 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)       : 2, 0.0065%
LFC < 0 (down)     : 0, 0%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



log2 fold change (MLE): condition_exporter_indiv_no_exporter_indiv exporter_GagZipMCP vs no_exporter_MS2 
Wald test p-value: condition exporter indiv no exporter indiv exporter GagZipMCP vs no exporter MS2 
DataFrame with 30877 rows and 6 columns
                          baseMean log2FoldChange     lfcSE       stat
                         <numeric>      <numeric> <numeric>  <numeric>
EGFP                   56992.42223     19.1483277  2.048627  9.3469064
IGHG1_ENSG00000211896    387.99351      2.9032525  0.436372  6.6531617
TSPAN6_ENSG00000000003  2843.73150     -0.0668959  0.221987 -0.3013504
TNMD_ENSG00000000005       7.14874      0.1942603  2.539770  0.0764874
DPM1_ENSG00000000419    1457.65253     -0.1424718  0.264270 -0.5391137
...                            ...            ...       ...        ...
ERCC-00164                       0             NA        NA         NA
ERCC-00165                       0             NA        NA         NA
ERCC-00168                       0         

In [14]:
# EPN24MCP vs no exporter (mCh-MS2x8)
res_no_exporter_indiv_EPN24MCP_vs_no_exporter <- results(dds_no_exporter_indiv_MS2, alpha=0.05, contrast=c("condition_exporter_indiv_no_exporter_indiv", "exporter_EPN24MCP", "no_exporter_MS2"))  # get results
res_no_exporter_indiv_EPN24MCP_vs_no_exporter <- res_no_exporter_indiv_EPN24MCP_vs_no_exporter[order(res_no_exporter_indiv_EPN24MCP_vs_no_exporter$padj),]  # sort by p value
summary(res_no_exporter_indiv_EPN24MCP_vs_no_exporter)
res_no_exporter_indiv_EPN24MCP_vs_no_exporter


out of 30785 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)       : 1, 0.0032%
LFC < 0 (down)     : 0, 0%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



log2 fold change (MLE): condition_exporter_indiv_no_exporter_indiv exporter_EPN24MCP vs no_exporter_MS2 
Wald test p-value: condition exporter indiv no exporter indiv exporter EPN24MCP vs no exporter MS2 
DataFrame with 30877 rows and 6 columns
                          baseMean log2FoldChange     lfcSE       stat
                         <numeric>      <numeric> <numeric>  <numeric>
EGFP                   56992.42223     18.6015824  2.048630  9.0800107
TSPAN6_ENSG00000000003  2843.73150     -0.0313754  0.221988 -0.1413380
TNMD_ENSG00000000005       7.14874      0.0381664  2.547077  0.0149844
DPM1_ENSG00000000419    1457.65253     -0.1031006  0.264265 -0.3901404
SCYL3_ENSG00000000457    597.22304      0.0964479  0.351298  0.2745471
...                            ...            ...       ...        ...
ERCC-00164                       0             NA        NA         NA
ERCC-00165                       0             NA        NA         NA
ERCC-00168                       0           

# Export results

In [15]:
output_dir <- "outs/transcriptome_stress/"

In [16]:
res_objects <- list(res_no_exporter_indiv_Gag_vs_no_exporter,
                    res_no_exporter_indiv_GagMCP_vs_no_exporter,
                    res_no_exporter_indiv_GagZipMCP_vs_no_exporter,
                    res_no_exporter_indiv_EPN24MCP_vs_no_exporter)

outfiles <- list("differential_expression_result_Gag_vs_no_exporter.csv",
                 "differential_expression_result_GagMCP_vs_no_exporter.csv",
                 "differential_expression_result_GagZipMCP_vs_no_exporter.csv",
                 "differential_expression_result_EPN24MCP_vs_no_exporter.csv")

In [17]:
for (i in 1:length(res_objects)) {
    res <- res_objects[i]
    outfile <- outfiles[i]
    outpath <- paste(output_dir, outfile, sep="")
    write.csv(as.data.frame(res), file=outpath)
}